### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [1]:
! pip install implicit

     |████████████████████████████████| 1.1MB 2.7MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419477 sha256=acf5b47fe3137b9cddbc839d2d20dc2111a8e62df9157bfdf9cae069439bca19
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [2]:
! pip install lightfm

     |████████████████████████████████| 307kB 2.7MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707622 sha256=475aa2d32f56de0216cc653974c9aa0819a4233843bfcc0385b18d6f7ee907ca
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [3]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp
from typing import Union
from collections import Counter
from sklearn.neighbors import KDTree
import time
from lightfm.datasets import fetch_movielens

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [4]:
ratings = pd.read_csv('ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [5]:
movie_info = pd.read_csv('movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

In [6]:
movie_info

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


Explicit данные

In [7]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [8]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [9]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [10]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

In [11]:
print(user_item_csr.todense())

[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [ ]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [ ]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [ ]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '2315    Babe: Pig in the City (1998)',
 '584    Aladdin (1992)',
 '1526    Hercules (1997)',
 '2692    Iron Giant, The (1999)',
 '3817    Went to Coney Island on a Mission From God... ...',
 '360    Lion King, The (1994)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [ ]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [ ]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [ ]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [ ]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1182    Aliens (1986)',
 '2502    Matrix, The (1999)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1179    Princess Bride, The (1987)',
 '1892    Rain Man (1988)',
 '1884    French Connection, The (1971)',
 '847    Godfather, The (1972)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

# Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

## Обработка explicit данных

In [12]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [13]:
ratings.rating.describe()

count    1.000209e+06
mean     3.581564e+00
std      1.117102e+00
min      1.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [14]:
Counter(ratings.rating)

Counter({1: 56174, 2: 107557, 3: 261197, 4: 348971, 5: 226310})

В табличке есть все айдишники user-ов без пропусков (то есть, все числа от 1 до max --- айдишники).

In [15]:
all_users = np.unique(ratings["user_id"])
set(range(np.min(all_users), np.max(all_users) + 1)) - set(all_users)

set()

А вот каких-то фильмов в таблице нет.

In [16]:
all_movies = np.unique(ratings["movie_id"])
len(set(range(np.min(all_movies), np.max(all_movies) + 1)) - set(all_movies))

246

При этом множества айдишников в фильмах в двух таблицах совпадают.

In [17]:
len(set(np.unique(movie_info.movie_id)).intersection(set(all_movies))) == len(set(all_movies))

True

Заведем внутренние айдишники для пользователей и фильмов.

In [18]:
user2idx = dict(zip(all_users, range(len(all_users))))
idx2user = dict(zip(range(len(all_users)), all_users))

In [19]:
movie2idx = dict(zip(all_movies, range(len(all_movies))))
idx2movie = dict(zip(range(len(all_movies)), all_movies))

И заведем матрицу рейтинга во внутренних айдишниках.

In [20]:
users_idxs = [user2idx[user] for user in ratings["user_id"]]
movies_idxs = [movie2idx[movie] for movie in ratings["movie_id"]]
user_item = sp.coo_matrix((ratings["rating"], (users_idxs, movies_idxs)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

In [31]:
user_item_csr

<6040x3706 sparse matrix of type '<class 'numpy.longlong'>'
	with 1000209 stored elements in Compressed Sparse Row format>

In [44]:
sp.diags(user_item_csr.multiply(40).getrow(0).toarray()[0], offsets=0).toarray()

array([[200.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [42]:
user_item_csr.multiply(40).getrow(0).toarray()[0]

array([200,   0,   0, ...,   0,   0,   0])

## Реализация SVD

In [21]:
class SVD():
    
    def __init__(self, 
                 n_users: int, 
                 n_items: int, 
                 latent_dim: int = 128, 
                 max_iter: int = 1e6, 
                 eps: float = 1., 
                 check_every = 100,
                 check_rmse_on = 1000,
                 lr: float = 1e-2, 
                 reg: dict = dict(zip(['u', 'v', 'b_u', 'b_v'], [1e-2, 1e-2, 1e-1, 1e-1])), 
                 seed: int = 73):
        self.n_users = n_users
        self.n_items = n_items
        self.max_iter = max_iter
        self.check_every = check_every
        self.check_rmse_on = check_rmse_on
        self.eps = eps
        self.latent_dim = latent_dim
        self.lr = lr
        self.reg = reg
        self.seed = seed
        np.random.seed(self.seed)
        self.U = np.random.uniform(low=0, high=1/np.sqrt(latent_dim), 
                              size=(n_users, latent_dim))
        self.V = np.random.uniform(low=0, high=1/np.sqrt(latent_dim), 
                              size=(n_items, latent_dim))
        self.mu = None
        self.b_u = None
        self.b_v = None
        #self.b_u = np.zeros(shape=(n_users, ), dtype='float32')
        #self.b_v = np.zeros(shape=(n_items, ), dtype='float32')
        
    
    def fit(self, X: sp.csr.csr_matrix):
        assert self.n_users, self.n_items == X.shape
        
        self.X = X
        self.mu = np.mean(X.data)
        self.b_u = np.array(X.mean(axis=1))
        self.b_v = np.array(X.mean(axis=0)).reshape(-1, )

        Q = 10000 + self.eps
        cur_iter = 0
        rand_indexes = np.random.randint(X.nnz, size=(self.max_iter, ))
        coords = X.nonzero()
        i_coords = coords[0][rand_indexes]
        j_coords = coords[1][rand_indexes]
        values = X.data[rand_indexes]
        while cur_iter < self.max_iter and Q > self.eps:
            
            # Select random nonzero x_ij
            i = i_coords[cur_iter]
            j = j_coords[cur_iter]
            x_ij = values[cur_iter]
            
            # Calculate gradients
            error = np.dot(self.U[i, :], self.V[j, :]) + self.mu + self.b_u[i] + self.b_v[j] - x_ij
            du_ij = error * self.V[j, :] + self.reg['u'] * self.U[i, :]
            dv_ij = error * self.U[i, :] + self.reg['v'] * self.V[j, :]
            dmu = error
            db_u_i = error + self.reg['b_u'] * self.b_u[i]
            db_v_j = error + self.reg['b_v'] * self.b_v[j]
            
            # Update
            self.U[i, :] -= self.lr * du_ij
            self.V[j, :] -= self.lr * dv_ij
            self.mu -= self.lr * dmu
            self.b_u[i] -= self.lr * db_u_i
            self.b_v[j] -= self.lr * db_v_j
             
            
            if not cur_iter % self.check_every:
                # We calculate RMSE on self.check_rmse_on random user-item pair 
                # (with known rating score) 
                rand_indexes2check = np.random.randint(X.nnz, size=(self.check_rmse_on, ))
                nonzero_inds = X.nonzero()
                nonzero_i = nonzero_inds[0][rand_indexes2check]
                nonzero_j = nonzero_inds[1][rand_indexes2check]
                nonzero_values = X.data[rand_indexes2check]
                Q = 0
                for value, i, j in zip(list(nonzero_values), list(nonzero_i), list(nonzero_j)):
                    Q += (self.U[i, :] @ self.V[j, :].T + self.mu + 
                         self.b_u[i] + self.b_v[j] - value) ** 2
                Q = np.sqrt(Q / self.check_rmse_on)
                print(f'iter: {cur_iter}, RMSE value: {Q}')
            cur_iter += 1
        self.U_KDTree = KDTree(self.U)
        self.V_KDTree = KDTree(self.V)
        
    
    def get_similars_items(self, item_ids: Union[int, list, np.ndarray], num_neighbors: int = 10):
        if isinstance(item_ids, list) or isinstance(item_ids, np.ndarray):
            return self.V_KDTree.query(self.V[item_ids, :], k=num_neighbors, return_distance=False)   
        return self.V_KDTree.query([self.V[item_ids, :]], k=num_neighbors, return_distance=False)
    
    def get_similars_users(self, user_ids: Union[int, list, np.ndarray], num_neighbors: int = 10):
        if isinstance(item_ids, list) or isinstance(item_ids, np.ndarray):
            return self.U_KDTree.query(self.U[user_ids, :], k=num_neighbors, return_distance=False)   
        return self.U_KDTree.query([self.U[item_ids, :]], k=num_neighbors, return_distance=False)
    
    def get_history(self, user_id: int):
        return [it_id for it_id in self.X.getrow(user_id).nonzero()[1]]

    def get_recommends(self, user_id: int, num_recs: int = 10):
        rating = self.X.getrow(user_id)
        
        not_known_items = np.array(list(
            set(range(self.X.shape[1])) - set(rating.nonzero()[1])))
        score = (self.U[user_id, :] @ self.V[not_known_items, :].T + 
                 self.mu + self.b_u[user_id] + self.b_v[not_known_items])
        order = np.argsort(score)
        order = order[::-1]
        return not_known_items[order[: num_recs]]

## Применение SVD

In [23]:
%%time 
svd = SVD(n_users=user_item_csr.shape[0], n_items=user_item_csr.shape[1], 
          latent_dim=64, max_iter=int(5e6), check_every=int(1e5), check_rmse_on=2000, eps=0.)
svd.fit(user_item_csr)


iter: 0, RMSE value: [1.61153818]
iter: 100000, RMSE value: [0.98351851]
iter: 200000, RMSE value: [0.98396521]
iter: 300000, RMSE value: [0.95446969]
iter: 400000, RMSE value: [0.94177664]
iter: 500000, RMSE value: [0.90513]
iter: 600000, RMSE value: [0.92932844]
iter: 700000, RMSE value: [0.92962365]
iter: 800000, RMSE value: [0.9358738]
iter: 900000, RMSE value: [0.92402081]
iter: 1000000, RMSE value: [0.89610727]
iter: 1100000, RMSE value: [0.91280357]
iter: 1200000, RMSE value: [0.93196971]
iter: 1300000, RMSE value: [0.93113327]
iter: 1400000, RMSE value: [0.91578552]
iter: 1500000, RMSE value: [0.91284055]
iter: 1600000, RMSE value: [0.89900201]
iter: 1700000, RMSE value: [0.92119904]
iter: 1800000, RMSE value: [0.90358924]
iter: 1900000, RMSE value: [0.89962986]
iter: 2000000, RMSE value: [0.90119802]
iter: 2100000, RMSE value: [0.90539967]
iter: 2200000, RMSE value: [0.87164021]
iter: 2300000, RMSE value: [0.88890274]
iter: 2400000, RMSE value: [0.89633983]
iter: 2500000, RMSE

Посмотрим на фильмы, похожие на историю игрушек

In [24]:
similar_movies = svd.get_similars_items(movie2idx[1])[0]
similar_movies = [idx2movie[idx] for idx in similar_movies]

In [25]:
movie_info.set_index('movie_id').loc[similar_movies, :]

,name,category
movie_id,,
1,Toy Story (1995),Animation|Children's|Comedy
3114,Toy Story 2 (1999),Animation|Children's|Comedy
588,Aladdin (1992),Animation|Children's|Comedy|Musical
1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
595,Beauty and the Beast (1991),Animation|Children's|Musical
953,It's a Wonderful Life (1946),Drama
2090,"Rescuers, The (1977)",Animation|Children's
364,"Lion King, The (1994)",Animation|Children's|Musical
2085,101 Dalmatians (1961),Animation|Children's


In [26]:
hist = svd.get_history(user2idx[4])
hist = [idx2movie[movie] for movie in hist]
movie_info.set_index('movie_id').loc[hist, :]

,name,category
movie_id,,
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
480,Jurassic Park (1993),Action|Adventure|Sci-Fi
1036,Die Hard (1988),Action|Thriller
1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1198,Raiders of the Lost Ark (1981),Action|Adventure
1201,"Good, The Bad and The Ugly, The (1966)",Action|Western
1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1214,Alien (1979),Action|Horror|Sci-Fi|Thriller


In [27]:
recs = svd.get_recommends(user2idx[4], 10)
recs = [idx2movie[rec] for rec in recs]
movie_info.set_index('movie_id').loc[recs, :]

,name,category
movie_id,,
2905,Sanjuro (1962),Action|Adventure
858,"Godfather, The (1972)",Action|Crime|Drama
3022,"General, The (1927)",Comedy
527,Schindler's List (1993),Drama|War
1148,"Wrong Trousers, The (1993)",Animation|Comedy
912,Casablanca (1942),Drama|Romance|War
1178,Paths of Glory (1957),Drama|War
913,"Maltese Falcon, The (1941)",Film-Noir|Mystery
745,"Close Shave, A (1995)",Animation|Comedy|Thriller


# Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

Реализация статьи [Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf)

In [189]:
class ALS:
    def __init__(self, 
                 n_users: int, 
                 n_items: int, 
                 latent_dim: int = 128, 
                 num_epochs: int = 1e6, 
                 eps: float = 0., 
                 check_rmse_on = 1000,
                 alpha: int = 40,
                 lr: float = 1e-2, 
                 reg: float = 1e-2,
                 seed: int = 73):
        
        self.n_users = n_users
        self.n_items = n_items
        self.latent_dim = latent_dim
        self.eps = eps
        self.check_rmse_on = check_rmse_on
        self.num_epochs = num_epochs
        self.reg = reg
        self.seed = seed
        self.alpha = alpha
        np.random.seed(seed)

        self.U = np.random.uniform(low=0, high=1/np.sqrt(latent_dim), 
                              size=(n_users, latent_dim))
        self.V = np.random.uniform(low=0, high=1/np.sqrt(latent_dim), 
                              size=(n_items, latent_dim))
    
    def check_rmse(self):
        #check RMSE
        # We calculate RMSE on self.check_rmse_on random user-item pair 
        # (with known rating score) 
        rand_indexes2check = np.random.randint(self.X.nnz, 
                                               size=(self.check_rmse_on, ))
        nonzero_inds = self.X.nonzero()
        nonzero_i = nonzero_inds[0][rand_indexes2check]
        nonzero_j = nonzero_inds[1][rand_indexes2check]
        nonzero_values = self.X.data[rand_indexes2check]
        Q = 0
        for value, i, j in zip(list(nonzero_values), list(nonzero_i), list(nonzero_j)):
            Q += (self.U[i, :] @ self.V[j, :].T - value) ** 2
        Q = np.sqrt(Q / self.check_rmse_on)
        print(f'RMSE value: {Q}')
        return Q
    
    def fit(self, X: sp.csr.csr_matrix):
        assert self.n_users, self.n_items == X.shape

        self.X = X
        Xt = X.transpose()
        cur_epoch = 0               
        Q = self.check_rmse()
            
        while cur_epoch < self.num_epochs and Q > self.eps:
            print(f'Epoch: {cur_epoch + 1}')

            # loop on users
            VtV = self.V.T @ self.V
            for i in range(self.n_users):
                if not i % 1000:
                    print(f'Users: [{i}/{self.n_users}]')
                r_u = X.getrow(i).toarray()[0]
                C_u = sp.diags(1 + self.alpha * r_u, offsets=0)           
                p_u = r_u > 0
                VtC_uV = (VtV + self.V.T.dot(sp.csr_matrix.dot(C_u - sp.identity(self.n_items), self.V)))
                inv = np.linalg.inv(VtC_uV + self.reg * sp.identity(VtC_uV.shape[0]))
                self.U[i, :] = inv @ self.V.T @ C_u @ p_u
            print('^_^')
            
            #loop on items
            UtU = self.U.T @ self.U
            for j in range(self.n_items):
                if not j % 1000:
                    print(f'Items: [{j}/{self.n_items}]')
                r_v = Xt.getrow(j).toarray()[0]
                C_v = sp.diags(1 + self.alpha * r_v, offsets=0)
                p_v = r_v > 0
                UtC_vU = (UtU + self.U.T.dot(sp.csr_matrix.dot(C_v - sp.identity(self.n_users), self.U)))
                inv = np.linalg.inv(UtC_vU + self.reg * sp.identity(UtC_vU.shape[0]))
                self.V[j, :] = inv @ self.U.T @ C_v @ p_v
            
            cur_epoch += 1
            Q = self.check_rmse()
        self.U_KDTree = KDTree(self.U)
        self.V_KDTree = KDTree(self.V)
        
    def get_similars_items(self, item_ids: Union[int, list, np.ndarray], num_neighbors: int = 10):
        if isinstance(item_ids, list) or isinstance(item_ids, np.ndarray):
            return self.V_KDTree.query(self.V[item_ids, :], k=num_neighbors, return_distance=False)   
        return self.V_KDTree.query([self.V[item_ids, :]], k=num_neighbors, return_distance=False)
    
    def get_similars_users(self, user_ids: Union[int, list, np.ndarray], num_neighbors: int = 10):
        if isinstance(item_ids, list) or isinstance(item_ids, np.ndarray):
            return self.U_KDTree.query(self.U[user_ids, :], k=num_neighbors, return_distance=False)   
        return self.U_KDTree.query([self.U[item_ids, :]], k=num_neighbors, return_distance=False)
    
    def get_history(self, user_id: int):
        return [it_id for it_id in self.X.getrow(user_id).nonzero()[1]]

    def get_recommends(self, user_id: int, num_recs: int = 10):
        rating = self.X.getrow(user_id)
        
        not_known_items = np.array(list(
            set(range(self.X.shape[1])) - set(rating.nonzero()[1])))
        score = self.U[user_id, :] @ self.V[not_known_items, :].T
        order = np.argsort(score)
        order = order[::-1]
        return not_known_items[order[: num_recs]]




In [105]:
users_idxs = [user2idx[user] for user in ratings["user_id"]]
movies_idxs = [movie2idx[movie] for movie in ratings["movie_id"]]
user_item = sp.coo_matrix((np.ones_like(users_idxs), (users_idxs, movies_idxs)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

In [233]:
als = ALS(n_users=user_item.shape[0], n_items=user_item.shape[1], num_epochs=20, eps=1e-1, latent_dim=64, alpha=20)

In [234]:
als.fit(user_item_csr)

RMSE value: 0.750188829785412
Epoch: 1
Users: [0/6040]
Users: [1000/6040]
Users: [2000/6040]
Users: [3000/6040]
Users: [4000/6040]
Users: [5000/6040]
Users: [6000/6040]
^_^
Items: [0/3706]
Items: [1000/3706]
Items: [2000/3706]
Items: [3000/3706]
RMSE value: 0.28070557195825874
Epoch: 2
Users: [0/6040]
Users: [1000/6040]
Users: [2000/6040]
Users: [3000/6040]
Users: [4000/6040]
Users: [5000/6040]
Users: [6000/6040]
^_^
Items: [0/3706]
Items: [1000/3706]
Items: [2000/3706]
Items: [3000/3706]
RMSE value: 0.22548378600838517
Epoch: 3
Users: [0/6040]
Users: [1000/6040]
Users: [2000/6040]
Users: [3000/6040]
Users: [4000/6040]
Users: [5000/6040]
Users: [6000/6040]
^_^
Items: [0/3706]
Items: [1000/3706]
Items: [2000/3706]
Items: [3000/3706]
RMSE value: 0.21603888652460043
Epoch: 4
Users: [0/6040]
Users: [1000/6040]
Users: [2000/6040]
Users: [3000/6040]
Users: [4000/6040]
Users: [5000/6040]
Users: [6000/6040]
^_^
Items: [0/3706]
Items: [1000/3706]
Items: [2000/3706]
Items: [3000/3706]
RMSE value

In [235]:
similar_movies = als.get_similars_items(movie2idx[1])[0]
similar_movies = [idx2movie[idx] for idx in similar_movies]
movie_info.set_index('movie_id').loc[similar_movies, :]

,name,category
movie_id,,
1,Toy Story (1995),Animation|Children's|Comedy
3114,Toy Story 2 (1999),Animation|Children's|Comedy
34,Babe (1995),Children's|Comedy|Drama
1265,Groundhog Day (1993),Comedy|Romance
2355,"Bug's Life, A (1998)",Animation|Children's|Comedy
2396,Shakespeare in Love (1998),Comedy|Romance
1923,There's Something About Mary (1998),Comedy
2321,Pleasantville (1998),Comedy
1641,"Full Monty, The (1997)",Comedy


In [236]:
hist = als.get_history(user2idx[4])
hist = [idx2movie[movie] for movie in hist]
movie_info.set_index('movie_id').loc[hist, :]

,name,category
movie_id,,
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
480,Jurassic Park (1993),Action|Adventure|Sci-Fi
1036,Die Hard (1988),Action|Thriller
1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1198,Raiders of the Lost Ark (1981),Action|Adventure
1201,"Good, The Bad and The Ugly, The (1966)",Action|Western
1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1214,Alien (1979),Action|Horror|Sci-Fi|Thriller


In [237]:
recs = als.get_recommends(user2idx[4], 10)
recs = [idx2movie[rec] for rec in recs]
movie_info.set_index('movie_id').loc[recs, :]

,name,category
movie_id,,
1200,Aliens (1986),Action|Sci-Fi|Thriller|War
858,"Godfather, The (1972)",Action|Crime|Drama
1221,"Godfather: Part II, The (1974)",Action|Crime|Drama
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
457,"Fugitive, The (1993)",Action|Thriller
1291,Indiana Jones and the Last Crusade (1989),Action|Adventure
110,Braveheart (1995),Action|Drama|War
3703,Mad Max 2 (a.k.a. The Road Warrior) (1981),Action|Sci-Fi


# Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

# Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных